In [1]:
BESTANDSNAAM = 'NL-20190108.zip'

In [2]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

In [3]:
import gtfs_kit as gk
import geopandas as gpd
import shapely
import re
from shapely.geometry import Point
import folium

# Inladen data

In [4]:
# Read recipe inputs
input_folder = dataiku.Folder("1YFDah2D")
input_folder_path = input_folder.get_path()
# orgineel
#filename = input_folder_path + '/gtfs (1).zip'
# nieuwe versie
filename = input_folder_path + '/' + BESTANDSNAAM
# jaar 2020
#filename = input_folder_path + '/NL-20200115.zip'

In [5]:
hub_mapping = dataiku.Dataset("hupmapping_optimal")
hub_mapping_df = hub_mapping.get_dataframe()

In [6]:
stationsnamen = dataiku.Dataset("Stationsnamen")
stationsnamen_df = stationsnamen.get_dataframe()

# Evaluatie van de feed bestanden en kwaliteit van de data

In [7]:
gk.list_feed(filename)

,file_name,file_size
0,agency.txt,2775
1,calendar_dates.txt,1930144
2,feed_info.txt,155
3,routes.txt,182024
4,shapes.txt,110607775
5,stop_times.txt,791442482
6,stops.txt,5254081
7,transfers.txt,1566743
8,trips.txt,49409035


In [8]:
feed = gk.read_feed(filename, dist_units='km')
feed.describe()

/home/dataiku/dss/code-envs/python/jeroen/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3343: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,indicator,value
0,agencies,"[allGo (Keolis), Arriva, Bravo (Arriva), Bravo..."
1,timezone,Europe/Amsterdam
2,start_date,20190108
3,end_date,20190804
4,num_routes,2669
5,num_trips,661616
6,num_stops,66576
7,num_shapes,8174
8,sample_date,20190117
9,num_routes_active_on_sample_date,2482


In [9]:
feed.validate()

,type,message,table,rows
0,warning,Unrecognized column feed_id,feed_info,[]
1,warning,"Repeated pair (route_short_name, route_long_name)",routes,"[571, 1288, 1289, 1359, 1362, 1430, 1434, 1475..."
4,warning,Unrecognized column fare_units_traveled,stop_times,[]
5,warning,"Repeated pair (trip_id, departure_time)",stop_times,"[8992876, 7262369, 541259, 1052095, 7711099, 6..."
2,warning,Unrecognized column platform_code,stops,[]
3,warning,Stop has no stop times,stops,"[774, 3674, 4039, 5918, 6326, 8602, 9939, 1066..."
6,warning,Unrecognized column from_route_id,transfers,[]
7,warning,Unrecognized column to_route_id,transfers,[]
8,warning,Unrecognized column from_trip_id,transfers,[]
9,warning,Unrecognized column to_trip_id,transfers,[]


In [10]:
week = feed.get_first_week()
dates = [week[1:3]] # selectie van dagen

# De juiste data uit feed selecteren (in 2 stappen)

## 1  Alleen de interessante hubs

In [11]:
hubs = hub_mapping_df[['geometry','hub']].drop_duplicates()
hubs['geometry'] = hubs['geometry'].apply(shapely.wkt.loads)
hubs_geo = gpd.GeoDataFrame(hubs, geometry = 'geometry', crs = 'epsg:4326')
hubs_geo.geometry = hubs_geo.geometry.map(lambda polygon: shapely.ops.transform(lambda x, y: (y, x), polygon))

In [12]:
feed_beperkt = gk.miscellany.restrict_to_area(feed, hubs_geo)
feed_beperkter = gk.miscellany.restrict_to_dates(feed_beperkt, dates = dates[0])

### Voor checks kan met onderstaande cel een kaartje van één specifieke hub worden geplot

In [13]:
# 1 hub, kaartje
#geselecteerde_hub = '2.01'
#feed_beperktst = gk.miscellany.restrict_to_area(feed_beperkter, hubs_geo.loc[hubs_geo['hub'] == geselecteerde_hub])
#kaart = feed_beperkt.map_routes(feed_beperktst.routes.route_id.iloc[:], include_stops=True)
#output_folder = dataiku.Folder("xTiM0SgH")
#kaart.save(output_folder.get_path() + '/alleroutes'+geselecteerde_hub+'.html')
#kaart

### Of een kaartje van één route

In [14]:
# 1 route, kaarte
#kaart = feed_beperkt.map_routes(feed_beperkt.routes.route_id.iloc[:], include_stops=False)
#output_folder = dataiku.Folder("xTiM0SgH")
#kaart.save(output_folder.get_path() + '/alleroutes.html')
#kaart

In [15]:
#kaart = feed_beperkt.map_routes(feed_beperkt.routes.route_id.loc[feed_beperkt.routes.route_id == '62512'], include_stops=True)
#kaart

## 2 Selectie maken van daluren

In [16]:
trip_stats = feed_beperkter.compute_trip_stats()

In [17]:
trip_stats_beperkt = trip_stats.loc[(trip_stats['start_time'] >= '10:00:00') & (trip_stats['start_time'] < '15:00:00')]

In [18]:
route_stats = feed_beperkt.compute_route_stats(trip_stats_beperkt, dates = dates[0])

# Berekeningen met de route_stats (lijnfrequentie)

In [19]:
route_stats['freq'] = 60/route_stats['mean_headway']

### Voor checks, selecteren van specifieke routes

In [20]:
# route kenmerken
#route_stats.loc[route_stats['route_id'] == '62541']

In [21]:
# trip kenmerken van een route
#trip_stats_beperkt.loc[trip_stats_beperkt['route_id'] == '62529']

In [22]:
# stop kenmerken van een route
# gk.stops.get_stops(feed_beperkter, date = dates[0][0], route_ids = ['19015'])

In [23]:
#plotten stops
# stops_te_plotten = gk.stops.get_stops(feed_beperkter, date = dates[0][0], route_ids = ['62539'])

# stops_te_plotten = stops_te_plotten.loc[:, ['stop_id', 'stop_lat', 'stop_lon']]
# begineind = trip_stats_beperkt.loc[trip_stats_beperkt['route_id'] == '62539', ['start_stop_id','end_stop_id']]
# stops_te_plotten['kleur'] = np.nan
# stops_te_plotten.loc[stops_te_plotten['stop_id'].isin(begineind['start_stop_id']),'kleur'] = 'green'
# stops_te_plotten.loc[stops_te_plotten['stop_id'].isin(begineind['end_stop_id']),'kleur'] = 'red'
# stops_te_plotten.loc[stops_te_plotten['kleur'].isna(),'kleur'] = 'blue'  
                     
# stops_te_plotten['geometry'] = stops_te_plotten.apply(lambda x: Point(x['stop_lat'],
#                                        x['stop_lon']),
#                        axis = 1)
# stops_geo = gpd.GeoDataFrame(stops_te_plotten, geometry = 'geometry', crs = 'epsg:4326')
# kaart = folium.Map([52.32785, 4.664520000000001],#[52.4084118, 4.6606255],
#                  zoom_start=7,
#                  tiles='cartodbpositron')
# for index, row in stops_geo.iterrows():
#    folium.CircleMarker([row['stop_lat'],
#                   row['stop_lon']],
#                  popup=row['stop_id'],
#                  color=row['kleur']
#                       ).add_to(kaart)

# kaart

# Hubs koppelen aan route_ids

In [24]:
hub_koppeltabel = pd.DataFrame()
counter = 0
for index, row in hubs_geo.iterrows():
    stops_df = gk.stops.get_stops_in_area(feed_beperkter, hubs_geo.iloc[counter:counter+1])
    stops_df = stops_df[['stop_id']]
    stops_df['hub'] = row['hub']
    hub_koppeltabel = pd.concat([hub_koppeltabel, stops_df])
    counter += 1

In [25]:
# duurt lang (~242 routes)
route_koppeltabel = pd.DataFrame()
counter = 0
for route in trip_stats_beperkt['route_id'].unique():
    routestops_df = gk.stops.get_stops(feed_beperkter, date = dates[0][0], route_ids = [route])
    routestops_df = routestops_df[['stop_id']]
    routestops_df['route_id'] = route
    route_koppeltabel = pd.concat([route_koppeltabel,routestops_df])

In [26]:
routes_bij_hubs = hub_koppeltabel.merge(route_koppeltabel, on = 'stop_id', how = 'inner')

# Het bepalen van richtingen per stop (in 3 stappen)

## 1 Het bepalen van begin- en eindhaltes

In [28]:
trip_stats_beperkt.loc[:, 'num_stops_max'] = trip_stats_beperkt.groupby('route_id')['num_stops'].transform(max)

/home/dataiku/dss/code-envs/python/jeroen/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/dataiku/dss/code-envs/python/jeroen/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [29]:
# de schiphol uitzondering: bij schipholroutes alleen de langste ritten 
schipholroutes = routes_bij_hubs.loc[routes_bij_hubs['hub'] == '2.01', 'route_id'].tolist()

In [30]:
filter_welschiphol = trip_stats_beperkt['route_id'].isin(schipholroutes)
filter_nietschiphol = 1 - trip_stats_beperkt['route_id'].isin(schipholroutes)
filter_langeroute = trip_stats_beperkt['num_stops'] == trip_stats_beperkt['num_stops_max']

In [31]:
trip_stats_beperkt = trip_stats_beperkt.loc[(filter_nietschiphol) | ((filter_welschiphol) & (filter_langeroute))]

In [32]:
start_stops = trip_stats_beperkt.loc[
    trip_stats_beperkt['start_time'] >= '10:00:00', ['route_id','direction_id','start_stop_id']
]
start_stops.rename(columns = {'start_stop_id': 'stop_id'}, inplace = True)

In [33]:
end_stops = trip_stats_beperkt.loc[trip_stats_beperkt['end_time'] < '15:00:00', ['route_id','direction_id','end_stop_id']]
end_stops.rename(columns = {'end_stop_id': 'stop_id'}, inplace = True)

In [34]:
one_directional_stops = pd.concat([start_stops, end_stops], sort = False)

### Checks voor richtingbepaling

In [50]:
#one_directional_stops.loc[one_directional_stops['route_id'] == '62538']

In [35]:
#routes_bij_hubs.loc[routes_bij_hubs['route_id'] == '62539']

In [36]:
# check voor, aantal begin- en eindstops van elke route (echt veel opties!)
#one_directional_stops.drop_duplicates()['route_id'].value_counts()

## 2 De richtingen vaststellen voor elke route x hub x stop combinatie

In [37]:
def identificeer_stoprichting(hub, stop, route, stop_df = one_directional_stops):
    stops_van_route = stop_df.loc[stop_df['route_id'] == route]
    if stop in stops_van_route['stop_id'].tolist():
        return 1
    else:
        return 2

In [38]:
routes_bij_hubs['richtingen'] = routes_bij_hubs.apply(
    lambda X: identificeer_stoprichting(X['hub'], X['stop_id'], X['route_id']), axis = 1
)

In [39]:
frequentie_bij_hubs = routes_bij_hubs.merge(route_stats[['route_id', 
                                                         'freq', 
                                                         'route_type']],
                                           on = 'route_id',
                                           how = 'inner')

## 3 De data aggregeren zodanig dat elke route x hub combinatie één eigenschap heeft

### *Binnen één hub kan een route meerdere stops hebben (begin, eind, en een stop om alleen passagiers te droppen). Dit wisselt ook behoorlijk van trip op trip. Vandaar dat uiteindelijk is gekozen voor de min functie. Als er een stop tussenzit die bekend staat als 1 richting, dan is het genoeg om die route te zien als 1-richting route. Dit blijkt goed te werken* 

In [40]:
motherfile = frequentie_bij_hubs.groupby(['hub','route_id'])[['richtingen']].min()
motherfile['freq'] = frequentie_bij_hubs.groupby(['hub','route_id'])['freq'].mean()
motherfile['route_type'] = frequentie_bij_hubs.groupby(['hub','route_id'])['route_type'].max()
motherfile = motherfile.reset_index()

In [41]:
#check
#motherfile[motherfile['hub'] == '2.01']

# Koppelen van routenaamgegevens uit Motherfile

In [42]:
motherfile = motherfile.merge(route_stats[['route_id','route_short_name']],
                              on = 'route_id',
                              how = 'inner'
                             )

In [43]:
motherfile = motherfile.merge(feed_beperkt.routes[['route_id', 'route_long_name']],
                             on = 'route_id',
                             how = 'left')

### Voor checks, selecteren van specifieke routes

In [44]:
#motherfile

In [45]:
# Voor checks kun je hier op naam zoeken
#motherfile.loc[motherfile['route_long_name'].str.contains('Hoorn')]
#motherfile.loc[motherfile['route_long_name'].str.contains('chiphol')]

# Bepalen bus type (in 3 hoofdstappen)

Dit is een kwetsbaar onderdeel van de dataverwerking. Er vindt namelijk een koppeling plaats met een Excelbestand (Motherfile), waar de namen van buslijnen en de route_id's handmatig zijn bijgehouden. Er kunnen veel dingen in de koppeling misgaan. Vandaar dat er een aantal tussenstappen worden gemaakt waarmee er achteraf kan worden nagegaan hoe de koppeling is gegaan. Als er een buslijn uit het Excelbestand niet kan worden gekoppeld, wordt dit vastgesteld. 

## 1 Opschonen Motherfile

In [46]:
# Inladen
#bus_types = dataiku.Dataset("bus_types")
#bus_types_df = bus_types.get_dataframe()
tuf_bus_types = dataiku.Dataset("TUF_motherfile")
tuf_bus_types_df = tuf_bus_types.get_dataframe()

In [47]:
# Kolommen netjes
tuf_bus_types_df.rename(columns = {'route_id':'route_id_fake', 'route_id_1':'route_id'}, inplace = True)
tuf_bus_types_df.loc[tuf_bus_types_df['route_id'].isna(),'route_id'] = tuf_bus_types_df.loc[tuf_bus_types_df['route_id'].isna(), 'route_id_fake']

In [48]:
# De bustypen netjes
tuf_bus_types_df.loc[:,'Type'] = tuf_bus_types_df.loc[:,'Type'].fillna('irrelevant')
tuf_bus_types_df.loc[:,'Type'] = tuf_bus_types_df.loc[:,'Type'].replace({'Buurtbus':'Streekbus'})
tuf_bus_types_df.loc[:,'koppelnaam'] = \
tuf_bus_types_df.loc[:, 'route_short_name'].map(str) + tuf_bus_types_df.loc[:, 'route_long_name']

In [49]:
# Filteren: bepalen welke route echt relevant zijn
tuf_bus_types_relevant = tuf_bus_types_df.loc[tuf_bus_types_df['Type'] != 'irrelevant']
tuf_bus_types_relevant.loc[:, 'route_id'] = tuf_bus_types_relevant['route_id'].astype(str)
tuf_bus_types_relevant.loc[:, 'route_id'] = tuf_bus_types_relevant.loc[:, 'route_id'].astype(float).astype(int).astype(str)

In [50]:
# Koppelnaam samenstelllen: combinatie van short en long name
motherfile['koppelnaam'] = motherfile.loc[:, 'route_short_name'].map(str) + motherfile.loc[:, 'route_long_name']

## 2 Bepalen op welke manieren Motherfile te koppelen is aan de feed (in 2 stappen)

### 1 De koppelpogingen zelf: op id en op naam

In [51]:
op_id = tuf_bus_types_relevant.merge(motherfile, on = 'route_id', how = 'left',suffixes = ('_jelle','_ruw'))
op_naam = tuf_bus_types_relevant.merge(motherfile, on = 'koppelnaam', how = 'left',suffixes = ('_jelle','_ruw'))

In [52]:
op_id['telling'] = op_id.loc[:, 'koppelnaam_ruw'].notna()
op_naam['telling'] = op_naam.loc[:, 'route_id_ruw'].notna()

### 2 Het resultaat van de koppelpogingen samenbrengen

In [53]:
aantal_keer_gekoppeld = op_id[['route_id','telling']].merge(op_naam[['route_id_jelle','telling']],
                                                            left_on = 'route_id',
                                                            right_on = 'route_id_jelle',
                                                            how = 'inner',
                                                            suffixes = ('_op_id','_op_naam'))

In [54]:
aantal_keer_gekoppeld.fillna(0, inplace = True)
aantal_keer_gekoppeld['telling_totaal'] = (aantal_keer_gekoppeld.loc[:, 'telling_op_id']*1) + (aantal_keer_gekoppeld.loc[:, 'telling_op_naam']*1)
aantal_keer_gekoppeld = aantal_keer_gekoppeld.drop_duplicates()

### Voor checks, het succes van het koppelen op de twee manieren

In [55]:
# 29 koppelen niet, 154 wel (inmiddels koppelen 8 niet, 175 wel)
aantal_keer_gekoppeld['telling_totaal'].value_counts()

2    144
1     27
0     12
Name: telling_totaal, dtype: int64

In [56]:
# dit is het Motherfile bestand + de info of de lijnen koppelen
koppelsucces = tuf_bus_types_relevant.merge(aantal_keer_gekoppeld[['telling_totaal', 'telling_op_id', 'telling_op_naam', 'route_id']], on = 'route_id', how = 'left')

In [57]:
# Welke buslijnen uit Motherfile zijn niet terug te vinden in de deed? 
#koppelsucces.loc[koppelsucces['telling_totaal']==0]

## 3 Het daadwerkelijk koppelen van Motherfile aan de feed (in 3 stappen)

### 1 Selectie maken van routes die te koppelen zijn (apart voor route_id en naam)

In [58]:
op_id = koppelsucces.loc[koppelsucces['telling_op_id'] == True]
op_naam = koppelsucces.loc[(koppelsucces['telling_totaal'] == 1) & (koppelsucces['telling_op_naam'] == True)]

### 2 Het daadwerkelijke koppelen

In [59]:
routes_met_voervoerstype1 = motherfile.merge(op_id[['route_id',
                                                             'bus_type',
                                                             'Type',
                                                          'koppelnaam'
                                                            ]], on = 'route_id', how = 'inner',suffixes = ('_ruw','_jelle'))
routes_met_voervoerstype2 = motherfile.merge(op_naam[['route_id',
                                                             'bus_type',
                                                             'Type',
                                                                   'koppelnaam'
                                                            ]], on = 'koppelnaam', how = 'inner',suffixes = ('_ruw','_jelle'))

### 3 Samenbrengen van het resultaat van beide koppelingen

In [60]:
routes_met_voervoerstype = pd.concat([routes_met_voervoerstype1,routes_met_voervoerstype2], sort = False)

In [61]:
# daar waar koppeling niet lukte op route_id, de route_id opvullen met het resultaat van de koppeling op naam
routes_met_voervoerstype.loc[routes_met_voervoerstype['route_id'].isna(), 'route_id'] = \
routes_met_voervoerstype.loc[routes_met_voervoerstype['route_id'].isna(),'route_id_ruw']

In [62]:
# het eindresultaat van het koppelen weer terugbrengen naar de tabel met alle relevante data 
routes_met_voervoerstype_beter = motherfile.merge(routes_met_voervoerstype[['route_id', 'Type']],
                                                 on = 'route_id',
                                                 how = 'left')

In [63]:
motherfile = routes_met_voervoerstype_beter.drop_duplicates()

# Bepalen van alle vervoerstypen

In [64]:
vervoerstypes_vertaling = {3:'bus', 2:'trein', 0:'tram', 4:'pont', 1:'metro'}
motherfile.loc[:,'route_type'] = \
motherfile['route_type'].replace(vervoerstypes_vertaling)

In [65]:
def bepaal_bustype(data):
    if type(data['Type']) != float:
        return data['Type']
    else:
        return 'onbekend bustype'

def bepaal_treintype(data):
    try:
        if re.findall(r'(ntercity)', data['route_short_name']):
            return 'Intercity Trein'
        elif re.findall(r'(neltrein)', data['route_short_name']):
            return 'Intercity Trein'
        elif re.findall(r'(printer)', data['route_short_name']):
            return 'Sprinter Trein'
        elif re.findall(r'(toptrein)', data['route_short_name']):
            return 'Sprinter Trein'
        else:
            return 'onbekend treintype'
    except:
        return 'let op, gaat iets mis'

def bepaal_vervoer_subtype(data):
    if data['route_type'] == 'bus':
        return bepaal_bustype(data)
    elif data['route_type'] == 'trein':
        return bepaal_treintype(data)
    elif data['route_type'] == 'tram':
        return 'Tram'
    elif data['route_type'] == 'pont':
        return 'Pont'
    elif data['route_type'] == 'metro':
        return 'Metro/Lightrail'
    else:
        return ''

In [66]:
motherfile.loc[:, 'vervoerstype'] = motherfile.loc[:,:].apply(lambda rij: bepaal_vervoer_subtype(rij), axis = 1)

In [67]:
# Als het vervoerstype een onbekend bustype is, kan deze worden verwijderd (kan bijv. een tijdelijke ns-bus zijn)
motherfile = motherfile.loc[motherfile['vervoerstype'] != 'onbekend bustype']

In [68]:
#motherfile.loc[motherfile['route_short_name'] == '21']

### Voor checks, in het eindbestand

In [69]:
# Voor welke lijnen is er geen frequentie vastgesteld (te weinig trips)? 
#motherfile.loc[motherfile['freq'].isna()]

# Het herstructureren van het eindresultaat: tabel in de juiste vorm

### Draaien van tabel

In [70]:
motherfile_pivot = motherfile.pivot_table(values='freq',
    index=['hub','richtingen'],
    columns='vervoerstype',
    aggfunc='sum',
                      fill_value = 0)

In [71]:
motherfile_pivot.reset_index(inplace = True)

### Toevoegen van station/hubnamen

In [72]:
motherfile_pivot = motherfile_pivot.merge(stationsnamen_df,
                                         left_on = 'hub',
                                         right_on = 'Code',
                                         how = 'left'
                                         )

In [73]:
motherfile_pivot.drop(columns = 'hub', inplace = True)

### Hernoemen en toevoegen van kolomnamen

In [74]:
motherfile_pivot.rename(
    columns = {
        'onbekend treintype': 'HSL trein', 
        'Intercity Trein': 'Intercity trein',
        'Sprinter Trein': 'Sprinter trein',
        'Metro/Lightrail': 'Metro/lightrail',
        'Streekbus': 'HOV/streekbus'     
    },
    inplace = True
)

In [75]:
motherfile_pivot.loc[:, 'R-net'] = 0
motherfile_pivot.loc[:, '(Inter)nationaal bus'] = 0

### Aanpassen van waarden van het attribuut 'richtingen'

In [76]:
motherfile_pivot.loc[:, 'richtingen'] = motherfile_pivot.loc[:, 'richtingen'].replace(
    {1 : 'Een-richting', 2 : 'twee-richting'})

### Aanpassen van hubcode

In [77]:
motherfile_pivot['Code'] = motherfile_pivot['Code'].astype(str)

### Nog een keer heen en weer terug draaien van tabel voor perfecte resultaat

In [78]:
melted = motherfile_pivot.melt(
    id_vars = ['Code', 'Station','richtingen'],
    value_vars = [
        'Stadsbus',
        'HSL trein',
        'R-net',         
        'Intercity trein', 
        '(Inter)nationaal bus',        
        'Metro/lightrail', 
        'HOV/streekbus',         
        'Sprinter trein', 
        'Tram', 
        'Pont' 
    ]   
)

In [79]:
eindresultaat = melted.pivot_table(
    values = ['value'],
    columns = ['richtingen', 'variable'],
    index = ['Code', 'Station']
).reset_index()

In [80]:
eindresultaat.fillna(0, inplace = True)

# Het wegschrijven van het eindresultaat

In [111]:
# Eindresultaat
vo_data = dataiku.Dataset("VO_data")
vo_data.write_with_schema(motherfile)

526 rows successfully written (w1sGm3u03B)


In [112]:
# Koppelevaluatie
koppel_evaluatie = dataiku.Dataset("koppel_evaluatie")
koppel_evaluatie.write_with_schema(koppelsucces)

183 rows successfully written (4q3qKWthkr)


In [113]:
# Eindresultaat in juiste vorm (als het niet nauw zou luisteren)
#vo_data_pivot = dataiku.Dataset("VO_data_pivot")
#vo_data_pivot.write_with_schema(motherfile_pivot)

## Toch nog een oplossing moeten maken voor csv in juiste vorm voor app

In [117]:
eerste_regel = ';;Een-richting;;;;;;;;;;twee-richting;;;;;;;;;\n'

In [118]:
tweede_regel = 'Code;Station'
for kolomnaam in eindresultaat.columns.get_level_values(2):
    if kolomnaam != '':
        tweede_regel += ';'
        tweede_regel += kolomnaam
tweede_regel += '\n'

In [119]:
output_folder = dataiku.Folder("2xAuVQdW")
file = open(output_folder.get_path() + '/GTFS 2019.csv', "w")
file.write(eerste_regel)
file.write(tweede_regel)
file.close()
eindresultaat.to_csv(
    output_folder.get_path() + '/GTFS 2019.csv', 
    sep = ';', decimal = ',', 
    index_label = False, 
    index = False, 
    header = False,
    mode = 'a'
)